# QURI Chemistry: a Qiskit Function by QunaSys

{/* cspell:ignore QSCI, CCSD, UCCSD, imags, CCGSD, CISD */}

<Admonition type="note">
Qiskit Functions are an experimental feature available only to IBM Quantum&trade; Premium Plan users. They are in preview release status and subject to change.
</Admonition>

## Overview

This function helps you solve the quantum chemistry ground state estimation problem by using one of two algorithms based on the [Quantum Selected Configuration Interaction (QSCI) algorithm](https://arxiv.org/pdf/2302.11320):

- QSCI
- Optimization-based QSCI

The overall procedure for using this functions is summarized in the flow chart below:

![Workflow](/images/guides/qunasys-qsci/workflow.svg)

## Function description

The QSCI algorithm samples from a specific ansatz on quantum computers. Each sampled bitstring corresponds to an electron configuration, and the number of time a bitstring is sampled represents the importance of that electron configuration. Choose a number, $R$, to select the $R$ most important electron configurations from the sample. The function constructs and diagonalizes the subspace Hamiltonian, depending on your choice of $R$. The smallest eigenvalue is termed "QSCI energy", which is the estimation of the QSCI algorithm's true ground state energy.

One important QSCI ingredient is its initial state preparations. Several possible initial state preparations are provided in this function. These include the [hardware efficient](https://www.nature.com/articles/nature23879), [UCCSD](https://www.nature.com/articles/srep03589), [kUpCCGSD](https://arxiv.org/abs/1810.02327), and [kuCJ](https://arxiv.org/abs/1909.12410) ansatzes. A CCSD amplitude-based double excitation ansatz is provided specifically to be executed on near-term devices. Ansatz-specific settings are also provided for you to customize according to your needs.

In addition to returning the QSCI energy, the function also computes the estimated ground state wave function to help you identify the important electron configurations.

## Get started

First, authenticate using your [IBM Quantum API token](http://quantum.ibm.com/), and select the Qiskit Function as follows:

In [1]:
from qiskit_ibm_runtime import QiskitRuntimeService
from qiskit_ibm_catalog import QiskitFunctionsCatalog

token = QiskitRuntimeService.saved_accounts()["default-ibm-quantum"]["token"]
catalog = QiskitFunctionsCatalog(token=token)

function = catalog.load("qunasys/quri-chemistry")

Call the function with the following arguments:

```python
function.run(
    method = ...,       # Allow only "QSCI" or "OPT_QSCI"
    parameters = {...}, # A JSON dictionary representing all the algorithm setting
    instance = ...,     # Your instance string.
    backend = ...       # The backend name you want to use.
)
```

A detailed description of each argument is summarized in the following table.

| Name       | Type | Description                        | Required | Default                             | Example           |
| :--------- | :--- | :--------------------------------- | :------- | :---------------------------------- | :---------------- |
| method     | str  | The algorithm name                 | Yes      | -                                   | “QSCI”            |
| parameters | json | Details settings of the algorithm | Yes      | -                                   |                   |
| instance    | str  | Your instance name                 | No       | Your default instance saved on disk | “ibm-q/open/main” |
| backend    | str  | The backend name you want to use   | No       | The least busy one available        | “ibm_torino”      |

## Set up the molecule

Here, you will learn how to build a molecule. The input is exactly the same as the `pyscf.gto.M` with an additional `active_space` option. The following summarizes the detailed settings for configuring a molecule.

| Name         | Type  | Description                                                                                                                                                      | Required | Default  | Example                                                 |
| :----------- | :---- | :--------------------------------------------------------------------------------------------------------------------------------------------------------------- | :------- | :------- | :------------------------------------------------------ |
| atom         | str   | The list of atom coordinates                                                                                                                                     | Yes      | -        | “H 0 0 0; H 0 0 1”                                      |
| basis        | str   | The basis set to represent the electronic wave function. Allowed values are the same those supported by [PySCF](https://pyscf.org/_modules/pyscf/gto/basis.html) | No       | “sto-3g” | “sto-3g”                                                |
| spin         | float | The $s_z$ quantum number of your molecule. Only spin=0.0 is supported                                                                                            | No       | 0.0      | 0.0                                                     |
| charge       | int   | The total charge of your molecule                                                                                                                                | No       | 0        | 0                                                       |
| active_space | json  | The active space you want to choose. See the “active space” table that follows for more information                                                                     | No       | None     | See the “active space” table that follows for more information |


The active space settings are summarized below.

| Name                | Type      | Description                                | Required | Default | Example   |
| :------------------ | :-------- | :----------------------------------------- | :------- | :------ | :-------- |
| n_active_ele        | int       | The number of active electrons             | Yes      | -       | 4         |
| n_active_orb        | int       | The number of active spatial orbitals      | Yes      | -       | 3         |
| active_orbs_indices | list[int] | The list of active spatial orbital indices | No       | None    | [0, 1, 2] |


As an example, you can configure the setting for a $H_2O$ molecule in JSON format:

In [3]:
mole = {
    "atom": "O 0 0 0; H 0.2774 0.8929 0.2544; H 0.6068 -0.2383 -0.7169",
    "basis": "6-31g",   # default to "sto-3g"
    "spin": 0.0,        # default to 0.0. Current code forces spin = 0.0
    "charge": 0,        # default to 0
    "active_space": {   # default to None. Equivalent to no active space chosen.
        "n_active_ele": 10,
        "n_active_orb": 10,
        "active_orbs_indices": [i for i in range(10)]
    }
}

## Execute algorithms

All the function input is in JSON format. You will set up a JSON string that specifies the options for running an algorithm. As stated previously, the two types of algorithms provided are "QSCI" and "OPT_QSCI".

### QURI Chemistry

In QURI Chemistry, you need to specify an ansatz, its settings, the state preparation method, and the QSCI setting. Typical QSCI input example:

```
algo_input = {
    "mole": mole,
    "ansatz": ...,
    "state_prep_method": ...,
    "ansatz_setting": ...,
    "qsci_setting": {"n_shots": ..., "number_of_states_pick_out": ...},
    "mitigation_setting": {}
},

```

| Name               | Type           | Description                                                                                 | Required | Default | Example                                                         |
| :----------------- | :------------- | :------------------------------------------------------------------------------------------ | :------- | :------ | :-------------------------------------------------------------- |
| mole               | json           | The molecule setting. See the table for setting molecules above                             | Yes      |         | The molecule setting. See the table for setting molecules above |
| ansatz             | str            | The name of the ansatz you want to use. Only “DoubleExcitation” and “UCCSD” are supported. `DoubleExcitation` is highly recommended to obtain a stable result, especially for systems larger than eight qubits   | Yes      |         | “DoubleExcitation”                                              |
| state_prep_method  | str            | The way you want to prepare the circuit parameter of your ansatz. Only CCSD is allowed | No       | “CCSD"  | “CCSD”                                                          |
| ansatz_setting     | str            | Specific setting you want to configure your ansatz with                                     | No       | None    | See the following examples                                          |
| qsci_setting       | dict[str, int] | Specify the number of shots and the size of the subspace. See the following section for details                                                       | Yes      |         | See the QSCI setting table                                      |
| mitigation_setting | json           | Settings for configuring error mitigation                                                   | No       | None    | See the mitigation section below                                |

#### QSCI algorithm parameter

The parameters for setting the QSCI algorithm is summarized in this table.

| Name                      | Type | Description                                                                                                                                                                          | Required | Example |
| :------------------------ | :--- | :----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------- | :------- | :------ |
| n_shots                   | int  | The total number of shots to sample from the ansatz                                                                                                                                  | Yes      | 100000  |
| number_of_states_pick_out | int  | Represents the size of the subspace Hamiltonian $R$. It affects the _classical_ resource, where _classical_ diagonalization of the $R \times R$ subspace Hamiltonian is performed if the number of bitstrings ($n$) sampled from the ansatz is larger than $R$. Otherwise, the $n \times n$ subspace matrix is diagonalized | Yes      | 10000   |

This example demonstrates the following setting:

In [4]:
qsci_setting = {"n_shots": 1e5, "number_of_states_pick_out": 1e4}

#### QURI Chemistry with UCCSD

UCCSD is a common chemistry-inspired ansatz. In the QURI Chemistry function, you can customize it with the `ansatz_setting` key. Options for customizing UCCSD is summarized in the table.

| Name             | Type | Description                                                                                | Required | Default | Example |
| :--------------- | :--- | :----------------------------------------------------------------------------------------- | :------- | :------ | :------ |
| use_singles      | bool | If True, all the single excitation modes are included. Otherwise, they are all turned off | No       | True    | True    |
| n_trotter        | int  | The number of Trotter steps for the exponentiated excitation operator                     | No       | 1       | 1       |
| reduce_parameter | bool | Only True is allowed for QSCI. refer to the `singlet_excitation` field in the [QURI Parts documentation](https://quri-parts.qunasys.com/api/quri_parts/openfermion/quri_parts.openfermion.ansatz#quri_parts.openfermion.ansatz.TrotterUCCSD)                                                            | No       | True    | True    |

A dictionary for running QURI Chemistry with UCCSD:

In [ ]:
qsci_uccsd_json = {
    "mole": mole,
    "ansatz": "UCCSD",
    "state_prep_method": "CCSD",    # this is optional and defaulted to "CCSD".
    "ansatz_setting": {"use_singles": False},   # this is optional and defaulted to the list above.
    "qsci_setting": {"n_shots": 1e5, "number_of_states_pick_out": 1e4},
}

#### QURI Chemistry with double excitation ansatz

When `DoubleExcitation` is chosen as the ansatz, you can specify how many excitation amplitudes to include with the `n_amplitudes` key in the `ansatz_setting` field. The default value is 10. The function sorts the CCSD amplitudes according to their magnitude and chooses the largest `n_amplitudes`  for constructing the ansatz. An example JSON dictionary using this ansatz follows:

In [5]:
qsci_double_exc_json = {
    "mole": mole,
    "ansatz": "DoubleExcitation",
    "state_prep_method": "CCSD",    # this is optional and defaulted to "CCSD".
    "ansatz_setting": {"n_amplitudes": 20}, # this is optional and defaulted to {"n_amplitudes": 10}.
    "qsci_setting": {"n_shots": 1e5, "number_of_states_pick_out": 5e4},
    "mitigation_setting": {     # please refer to the "error mitigation" section of this tutorial.
        "configuration_recovery": {
            "number_of_states_pick_out": 10000
        }
    }
}

#### Execute QSCI

Example of running the `QSCI` algorithm:

In [30]:
import datetime
import time
job = function.run(
    method="QSCI",
    parameters=qsci_double_exc_json,
    instance="INSTANCE",
    backend="ibm_strasbourg",
)

while True:
   print(datetime.datetime.now(), job.status())
   if job.status() in ["ERROR", "DONE"]:
       break
   time.sleep(10)

result = job.result()

2024-08-27 23:01:36.090958 QUEUED
2024-08-27 23:01:49.358961 QUEUED
2024-08-27 23:02:02.713788 QUEUED
2024-08-27 23:02:15.481531 RUNNING
2024-08-27 23:02:28.200317 RUNNING
2024-08-27 23:02:41.446378 RUNNING
2024-08-27 23:02:54.563686 RUNNING
2024-08-27 23:03:07.706745 RUNNING
2024-08-27 23:03:20.400988 RUNNING
2024-08-27 23:03:33.708046 RUNNING
2024-08-27 23:03:46.656525 RUNNING
2024-08-27 23:03:59.385160 RUNNING
2024-08-27 23:04:12.528769 RUNNING
2024-08-27 23:04:25.625868 RUNNING
2024-08-27 23:04:38.718299 RUNNING
2024-08-27 23:04:51.828514 RUNNING
2024-08-27 23:05:04.745142 RUNNING
2024-08-27 23:05:17.727985 RUNNING
2024-08-27 23:05:31.061746 RUNNING
2024-08-27 23:05:43.821970 RUNNING
2024-08-27 23:05:56.897187 RUNNING
2024-08-27 23:06:10.088404 RUNNING
2024-08-27 23:06:22.959995 RUNNING
2024-08-27 23:06:35.873720 RUNNING
2024-08-27 23:06:48.798231 RUNNING
2024-08-27 23:07:02.731781 RUNNING
2024-08-27 23:07:15.819290 RUNNING
2024-08-27 23:07:29.360154 RUNNING
2024-08-27 23:07:42.341

You can print out the result from the run:

In [41]:
print(result)

{'qsci_energy': -76.05077853515165, 'state_vector': {'bits': [1023, 131583, 3039, 3063, 2943, 33663, 263163, 9087, 2043, 33759, 66303, 525183, 263151, 17403, 2031, 525303, 263103, 9207, 1983, 5115, 33783, 9183, 5055, 5103, 17343, 17391, 525279, 139647, 70383, 18411, 67311, 164319, 139767, 655839, 164343, 655743, 267243, 527319, 6123, 533367, 133623, 11127, 41847, 279531, 82683, 67263, 533343, 82671, 133503, 264171, 11103, 35703, 41823, 67323, 527199, 164223, 133599, 35679, 393855, 328383, 533463, 264111, 328431, 279483, 267183, 132735, 21483, 139743, 557919, 328443, 265083, 18351, 41943, 35799, 132591, 557943, 141687, 11223, 197595, 21423, 50043, 527223, 655863, 21435, 279471, 558039, 82623, 6075, 6063, 98943, 70335, 70395, 141663, 25455, 265179, 10107, 6399, 267195, 393723, 148191, 197583, 197439, 7131, 18363, 295923, 3963, 535383, 265203, 25407, 526239, 329403, 264447, 590331, 196863, 270591, 688503, 526287, 344763, 25467, 19359, 787323, 148095, 132855, 43863, 34047, 393951, 265071, 

Use the \"qsci_energy\" key to see the QSCI energy:

In [48]:
CASCI_ENERGY = -76.0508056368444    # Exact result
qsci_energy = result["qsci_energy"]
print(f"CASCI energy: {CASCI_ENERGY} Ha")
print(f"QSCI energy: {qsci_energy} Ha")
print(f"Energy error: {(qsci_energy - CASCI_ENERGY) * 1000: .2e} mHa")

CASCI energy: -76.0508056368444 Ha
QSCI energy: -76.05077853515165 Ha
Energy error:  2.71e-02 mHa


The returned ground state energy estimation of an $\text{H}_2 \text{O}$ molecule is very close to the exact result!

The `state_vector` field contains three values: `bits`, `reals`, and `imags`. The `bits` value is a list of integers representing electron configurations. The $i$-th position of the `reals` (`imags`) field represents the real (imaginary) part of the amplitude of the electron configuration on position $i$. You can sort it by magnitude and print out the top 10 most important electron configurations.

In [66]:
import numpy as np

amps = np.array(result["state_vector"]["reals"]) + 1j * np.array(result["state_vector"]["imags"])
sort_idx = np.argsort(np.abs(amps))

for i, (bit, real, imag) in enumerate(zip(
    np.array(result["state_vector"]["bits"])[sort_idx][::-1],
    np.array(result["state_vector"]["reals"])[sort_idx][::-1],
    np.array(result["state_vector"]["imags"])[sort_idx][::-1],
)):
    if i > 20:
        break
    print(
        f"Electron configuration: {bin(bit)[2:].zfill(20)}, "
        f"amplitude: {real + 1j * imag: .3e}, "
        f"magnitude: {real**2 + imag**2: .3e}"
    )

Electron configuration: 00000000001111111111, amplitude: -7.671e-01+6.185e-01j, magnitude:  9.710e-01
Electron configuration: 00110000000011111111, amplitude:  4.492e-02-3.622e-02j, magnitude:  3.330e-03
Electron configuration: 00000011001111001111, amplitude:  3.379e-02-2.724e-02j, magnitude:  1.884e-03
Electron configuration: 00001100001111001111, amplitude:  2.247e-02-1.812e-02j, magnitude:  8.333e-04
Electron configuration: 00100001000111101111, amplitude: -2.172e-02+1.751e-02j, magnitude:  7.780e-04
Electron configuration: 00010010001011011111, amplitude: -2.171e-02+1.750e-02j, magnitude:  7.775e-04
Electron configuration: 00000001101101101111, amplitude: -2.006e-02+1.617e-02j, magnitude:  6.637e-04
Electron configuration: 00000010011110011111, amplitude: -2.005e-02+1.617e-02j, magnitude:  6.636e-04
Electron configuration: 00000000111100111111, amplitude:  1.980e-02-1.596e-02j, magnitude:  6.467e-04
Electron configuration: 11000000001111001111, amplitude:  1.950e-02-1.572e-02j, ma

### Optimization-based QSCI

If you don't want to use CCSD as the state preparation method, or you want to optimize further from the CCSD initial state, you can use the optimization-based QSCI. This algorithm gives you access to more chemistry-inspired ansatz with random initial parameters. The following is a table of ansatz and their corresponding supported state preparation methods:


| Ansatz                | Preparation method    |   Ansatz Setting Default  | Note                                                  |
| --------------------- | --------------------- | ------------------------- | ----------------------------------------------------- |
| Double Excitation     | CCSD                  | `{n_amplitudes: 10}`          | Lets you optimize from the result in the last example    |
| UCCSD                 | CCSD (default), Random          | `{use_singles: True, n_trotter: 1, reduce_parameter: True}` | Not recommended for systems with eight or more qubits      |
| KuCJ                  | Random                | `{k: 1}`                          | Recommended for system up to 16 qubits                 |
| KUpCCGSD              | Random                | `{k: 1, n_trotter: 1, reduce_parameter: False} `                         |                                                       |
| Hardware Efficient    | Random                | `{n_layers: depend on qubit size}        `                  |                                                       |

If you don't specify the preparation method and the ansatz setting, default values are used. For this algorithm, the QSCI energy is used as the cost function for the COBYLA optimizer. In each iteration, the circuit parameters are updated and the QSCI energy is evaluated with the circuit carrying the new set of circuit parameters.

The JSON dictionary used to run this algorithm is similar to that of QSCI. It contains one additional field, "max_iter", for limiting the resource usage. The default is 2000 iterations.

The follwing is an example of running this algorithm with the 1-uCJ ansatz:

In [86]:
opt_qsci_1ucj_json = {
    "mole": mole,
    "ansatz": "KuCJ",
    "state_prep_method": "RANDOM",  # this is optional and defaulted to "RANDOM" for "KuCJ".
    "ansatz_setting": {"k": 1},     # this is optional and defaulted to the table above.
    "qsci_setting": {
        "n_shots": 1e5, "number_of_states_pick_out": 5e4
    },
    "max_iter": 5,                  # Default to 2000. Choose 5 iteration for demonstration.
    "mitigation_setting": {         # Refer to the "error mitigation" section.
        "configuration_recovery": {
            "number_of_states_pick_out": 10000
        }
    }
}

In [ ]:
import time
opt_job = function.run(
    method="OPT_QSCI",
    parameters="opt_qsci_1ucj_json",
    instance="ibm_strasbourg",
)
start = time.time()

while True:
    print(datetime.datetime.now(), opt_job.status())
    if opt_job.status() in ["ERROR", "DONE"]:
        break
    time.sleep(10)

opt_result = opt_job.result()

You can print out the result.

In [80]:
print(opt_result)

{'qsci_energy': -76.05051672600099, 'state_vector': {'bits': [1023, 1983, 2943, 33783, 3039, 263151, 33663, 2031, 5103, 9087, 525183, 33759, 2043, 17343, 9207, 9213, 525279, 17406, 131583, 5115, 525303, 263163, 525309, 17391, 9183, 3063, 263103, 66303, 3069, 2046, 17403, 263166, 5118, 33789, 5055, 67323, 133503, 70335, 264123, 655863, 67263, 279471, 139743, 6138, 67311, 655743, 133599, 139647, 11127, 164223, 267195, 527223, 18411, 18426, 533343, 558039, 82671, 21486, 264111, 267183, 11223, 328443, 328446, 70383, 328431, 35799, 533373, 6063, 527199, 11103, 6126, 21483, 533469, 18363, 279534, 67326, 41847, 533463, 279486, 655839, 533367, 133629, 70398, 11229, 21498, 558045, 11133, 82683, 21423, 41823, 139767, 527319, 35679, 557919, 264126, 164343, 10095, 35703, 267243, 655869, 267198, 21435, 328383, 279531, 527325, 271356, 6075, 18366, 83631, 164349, 264171, 557943, 271311, 83691, 139773, 527229, 6078, 133623, 13215, 82686, 6123, 68319, 41949, 279546, 35805, 21438, 18414, 270591, 279483,

The returned result has the same fields as the one in the QSCI example.

In [91]:
CASCI_ENERGY = -76.0508056368444    # Exact result
opt_qsci_energy = opt_result["qsci_energy"]
print(f"CASCI energy: {CASCI_ENERGY} Ha")
print(f"QSCI energy: {opt_qsci_energy} Ha")
print(f"Energy error: {(opt_qsci_energy - CASCI_ENERGY) * 1000: .2e} mHa")
print()
print("Top 20 important configurations:")
amps = np.array(opt_result["state_vector"]["reals"]) + 1j * np.array(opt_result["state_vector"]["imags"])
sort_idx = np.argsort(np.abs(amps))

for i, (bit, real, imag) in enumerate(zip(
    np.array(opt_result["state_vector"]["bits"])[sort_idx][::-1],
    np.array(opt_result["state_vector"]["reals"])[sort_idx][::-1],
    np.array(opt_result["state_vector"]["imags"])[sort_idx][::-1],
)):
    if i > 20:
        break
    print(
        f"Electron configuration: {bin(bit)[2:].zfill(20)}, "
        f"amplitude: {real + 1j * imag: .3e}, "
        f"magnitude: {real**2 + imag**2: .3e}"
    )

CASCI energy: -76.0508056368444 Ha
QSCI energy: -76.05051672600099 Ha
Energy error:  2.89e-01 mHa

Top 20 important configurations:
Electron configuration: 00000000001111111111, amplitude: -1.934e-01-9.664e-01j, magnitude:  9.713e-01
Electron configuration: 00110000000011111111, amplitude:  1.127e-02+5.629e-02j, magnitude:  3.295e-03
Electron configuration: 00000011001111001111, amplitude:  8.434e-03+4.214e-02j, magnitude:  1.847e-03
Electron configuration: 00001100001111001111, amplitude:  5.640e-03+2.818e-02j, magnitude:  8.258e-04
Electron configuration: 00010010001011011111, amplitude: -5.445e-03-2.721e-02j, magnitude:  7.698e-04
Electron configuration: 00100001000111101111, amplitude: -5.418e-03-2.707e-02j, magnitude:  7.622e-04
Electron configuration: 00000010011110011111, amplitude: -5.053e-03-2.524e-02j, magnitude:  6.628e-04
Electron configuration: 00000001101101101111, amplitude: -5.023e-03-2.510e-02j, magnitude:  6.551e-04
Electron configuration: 00000000111100111111, amplit

## Error mitigation

Usually, the problem at hand needs to respect some symmetry, especially particle number and spin conservation. To restore symmetry, use symmetry post-selection or [configuration recovery](https://arxiv.org/pdf/2405.05068v1) error mitigation. You can choose either of them with the \"mitigation_setting\" field in either algorithm. Symmetry post-selection is the default value.

Post-selection removes the erroneous states completely, leaving only the basis states to construct the subspace Hamiltonian. This usually saves classical resources, but might miss many states. Configuration recovery attempts to flip individual bits in erroneous basis states with respect to some probability distribution. Using configuration recovery increases the number of basis states to build a larger subspace Hamiltonian, but it uses a lot more classical resources.


| Name                     | Type                        | Description                                                                                                                             | Required | Default |
| :----------------------- | :-------------------------- | :-------------------------------------------------------------------------------------------------------------------------------------- | :------- | :------ |
| particle_number_symmetry | bool                        | If True, remove bitstrings that do not respect particle number conservation from sampling result                                    | No       | TRUE    |
| spin_symmetry            | bool                        | If True, remove bitstrings that do not respect $s_z$ conservation from sampling result.                                              | No       | TRUE    |
| configuration_recovery   | Union[bool, dict[str, int]] | Activate configuration recovery if set to values other than False. Refer to the configuration recovery section for dict[str, int] input | No       | FALSE   |

### Symmetry post-selection

These keys control the symmetry post-selection: `particle_number_symmetry` and `spin_symmetry`. Setting `particle_number_symmetry` to True selects those states that conserve the total electron numbers.  Setting `spin_symmetry` to True selects those states that conserve the $s_z$ quantum number. Currently, only $s_z = 0$ is supported. The default is:

In [ ]:
mitigation_setting = {
    "particle_number_symmetry": True,
    "spin_symmetry": True,
}

<Admonition type="note">You cannot set spin symmetry to True when particle symmetry is False. </Admonition>

### Configuration recovery

Instead of the symmetry post-selection, you can also choose to do configuration recovery. The most general sets of configuration recovery are summarized in the table below.

| Name                      | Type        | Description                                                                                                          | Required | Default |
| :------------------------ | :---------- | :------------------------------------------------------------------------------------------------------------------- | :------- | :------ |
| n_recover_iteration       | int         | The number of recovery iterations                                                                                   | No       | 5       |
| n_batch                   | int         | The number of batches generated per iteration                                                                       | No       | 1       |
| number_of_states_pick_out | int or None | The dimension of the QSCI vector in the recovery process. If None, it is set to the same as the value used for QSCI | No       | None    |
| recovery_r_multiple       | int         | The multiplier for number_of_states_pick_out. This is the number of states selected from the raw sample             | No       | 10      |

To activate configuration recovery, specify `configuration_recovery = True` in the error mitigation field, or customize any of the above fields. For example:

In [ ]:
# Use default configuration recovery setting
default_mitigation_setting = {
    "mitigation_setting": {
        "configuration_recovery": True
    }
}

# Customized configuration recovery setting
customized_mitigation_setting = {
    "mitigation_setting": {
        "configuration_recovery": {
            "n_recover_iteration": 20,
            "n_batch": 10,
            "number_of_states_pick_out": 5000
        }
    }
}

<Admonition type="note">You cannot enable any other mitigation methods when `configuration_recovery` is turned on.</Admonition>

## Performance

### $\text{N}_2$ dissociation curve

As a performance benchmark, we provide the 20-qubit $\text{N}_2$ dissociation curve as a demonstration.

![N2_dissociation](/images/guides/qunasys-qsci/N2_dissociation.svg)

The red curve in the plot is generated by the QSCI method with the chosen ansatz `DoubleExcitation`. You can reproduce it with the following code using different choices of `d`, the distance between the nitrogen atoms.

In [ ]:
d = 1.0
n2_dissociation_qsci = {
    "mole": {"atom": f"N 0 0 0; N 0 0 {d}",},
    "ansatz": "DoubleExcitation",
    "state_prep_method": "CCSD",
    "qsci_setting": {
        "n_shots": 100000, "number_of_states_pick_out": 50000
    },
    "mitigation_setting": {
        "configuration_recovery": True
    },
}
job = function.run(
    method="QSCI",
    parameters=n2_dissociation_qsci,
    backend="ibm_strasbourg"
)

Note that each point typically takes about one minute of QPU usage. On the classical side, while $R$ is chosen to be 50000, it doesn't necessarily mean that diagonalization of a $50000 \times 50000$ matrix is done. The number of samples after configuration recovery in this case is approximately 5000 to 7000, which sets the dimension of the subspace Hamiltonian. Thus, the classical diagonalization cost is much smaller than that of typical full configuration interaction (FCI) computation.

The key takeaway of this result is that QSCI with suitable configuration recovery outperforms scalable classical methods such as Hartree-Fock (HF), CCSD, and CISD at _all_ choices of `d`. This implies that at a scale where FCI energy is no longer available, QSCI serves as a reliable method to estimate the ground state energy at all distances. For example, using the 6-31g basis,
a 36-qubit $\text{N}_2$ dissociation curve produced by QURI Chemistry is given by

![N2_dissociation](/images/guides/qunasys-qsci/36_qubit_N2_dissociation.svg)

Here, the FCI curve is not available on a typical laptop (M2 Pro Chip, 16G RAM). However, QSCI can produce a qualitatively correct dissociation curve compared to the classical methods on the same machine, with the assistance of the `ibm_strasbourg` quantum computer.

### Azobenzene

Azobenzene has two isomers, trans-azobenzene and cis-azobenzene. The energy difference between the ground states of the two isomers plays an important role in the photoisomerization of azobenzene. As an example, we benchmark the ground state energy difference between QSCI and FCI, with different active space settings up to 28 qubits. With configuration recovery, QURI Chemistry yields error under 5 mHa with only $10^5$ shots, 30 to 40 seconds of QPU usage, and under 10 minutes of classical post-processing time per point.

![Azobenzene](/images/guides/qunasys-qsci/azobenzene.svg)

## Support

For molecules larger than 20 qubits, please contact [sales@qunasys.com](mailto:sales@qunasys.com).

## Next steps

<Admonition type="tip" title="Recommendations">

- [Request access to QunaSys QURI Chemistry](https://quantum.ibm.com/functions?id=42f5a0ea-2c74-4681-a973-3ef97de97ee4)
- Try the [Compute dissociation curves for strong coupling systems with QunaSys QURI Chemistry](https://learning.quantum.ibm.com/tutorial/compute-dissociation-curves-for-strong-coupling-systems-with-quna-sys-qsci) tutorial.

</Admonition>